<a href="https://colab.research.google.com/github/sundarjhu/UACJ_Jornada2021/blob/main/Espectro_UACJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
__author__ = 'Aida Wofford <awofford@astro.unam.mx>, Sundar Srinivasan <s.srinivasan@irya.unam.mx>'
__version__ = '20210421'
__datasets__ = ['SDSS']  # datasets used in this notebook
__keywords__ = ['UACJ Workshop 2021', 'SDSS', 'spectrum'], # keywords relevant to this notebook

In [2]:
import warnings, numpy as np
warnings.filterwarnings("ignore", category = np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)
try:
  from astroquery.sdss import SDSS
except:
  !pip install astroquery
  from astroquery.sdss import SDSS

     |████████████████████████████████| 6.5MB 18.2MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 3.2MB 41.4MB/s 
  Created wheel for astroquery: filename=astroquery-0.4.1-cp37-none-any.whl size=3831874 sha256=226ef672ec6485a0dee7e352707907e8765479a1b48bf9fd4342cb9627911b4b
  Stored in directory: /root/.cache/pip/wheels/88/f8/b7/a254cd96e808f708bc0b7d755a8e095c56fbbe94099d7b464f
Successfully built astroquery


# Análisis del espectro óptico de KISSR 298

## Bajar el espectro de la base de datos del Sloan Digital Sky Survey (SDSS)

INSTRUCCIÓN. Nuestra galaxia se llama KISSR 298. Necesitamos las coordenadas (RA, Dec) o (longitude, latitude) del objeto en el sistema de referencia Equatorial (J2000.0). Encuentra las coordenadas buscando por al objeto por su nombre aquí:
https://ned.ipac.caltech.edu/.
Verifica que las coordenadas del objecto dadas abajo están correctas y de no ser así, modifícalas con los valores que encontraste.

In [3]:
from astropy import coordinates as coords

#We can search by coordinates (in degrees)
RA = 202.457480 #longitude in Equatorial J2000
DEC = 29.579716 #latitude in Equatorial J2000
pos = coords.SkyCoord(str(RA) + 'd ' + str(DEC) + 'd', frame = 'icrs') #units of degrees

#Or we can have the `coords` module automatically look up the coordinates from the name of the source 
pos = coords.SkyCoord.from_name("KISSR 298")

#Once the coordinates are known, we search the SDSS archive for spectra of objects around this position.
xid = SDSS.query_region(pos, spectro = True)
sp = SDSS.get_spectra(matches = xid, plate = 1978, mjd = 53473, fiberID = 371)[0]
#El espectro está en el segundo encabezado del archivo (sp[1])
specdata = sp[1].data

INSTRUCCIÓN. Para bajar el espectro de la base de datos, necesitamos los IDs de la placa, la fecha y la fibra con que se usaron para obtener los datos. Busca el objecto usando las coordenadas en: https://skyserver.sdss.org/dr14/en/tools/chart/chartinfo.aspx. Dále a "Get Image" para ver la imagen del objeto. Puedes cambiar el tamaño de la imagen con los botones de "+" y "-".

INSTRUCCIÓN. Haz click en "Explore" para ir a donde se encuentra el espectro. Abajo a la derecha, en donde dice "Interactive Spectrum", hay una tabla con información sobre las observaciones del objeto. Verifica que la siguiente información es correcta y de no ser así, modifícala.

In [4]:
# plate=1978
# mjd=53473 #Modified Julian Date
# fiberid=371

In [5]:
# canonical = 'https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid='+str(plate)+'&mjd='+str(mjd)+'&fiberid='+str(fiberid)

Importamos los paquetes necesarios.

In [6]:
from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from astropy.visualization import quantity_support
quantity_support();  # for getting units on the axes below

Los espectros están en archivos formato fits.

Cargamos los datos y contruimos los vectores de longitudes de onda (wave) , de flujos (flux), y de errores en los flujos (err).

In [7]:
# f = fits.open(canonical) 
#El espectro está en el segundo encabezado del archivo (f[1])
# specdata = f[1].data 
# f.close()

wave = 10**specdata['loglam'] * u.AA # longitud de onda
flux = specdata['flux'] * 10**-17 * u.Unit('erg cm-2 s-1 AA-1') # flujo
ivar = specdata['ivar']# inverso de la varianza
err = 1 / np.sqrt(ivar) # error * u.Unit('erg cm-2 s-1 AA-1') # error en el flujo

Instalamos el paquete specutils.

In [8]:
# Install a pip package in the current Jupyter kernel if it isn't already installed
import sys
try:
  from specutils import Spectrum1D
except:
  #!{sys.executable} -m pip install specutils
  !pip install specutils
  from specutils import Spectrum1D

     |████████████████████████████████| 645kB 16.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 512kB 47.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174kB 40.9MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for specutils: filename=specutils-1.2-cp37-none-any.whl size=177575 sha256=893db437bc5468f1c59cf22a74b5ff8bd6dae50b0e99c8b70287663f3c207654
  Stored in directory: /root/.cache/pip/wheels/af/88/32/7ccdd3bfdd9469747850b14674938ccc180caf58f69e006659
  Created wheel for asdf: filename=asdf-2.7.3-cp37-none-any.whl size=573206 sha256=4e3507c9056e0ea610c8c89663d61a9b00c3452c1d37ba1b75b55e54c1a0a4d7
  Stored in directory: /root/.cache/pip/wheels/01/cc/ff/c29ad7c212112bbdd9f416c24915607bdf2b4d83641f876e2e
Successf

ContextualVersionConflict: ignored

Vamos a poner los datos en el formato apropiado para "astropy" y crear un objeto llamado "Spectrum1D" que vamos a graficar.

In [ ]:
spec = Spectrum1D(spectral_axis=wave, flux=flux)

Graficamos el espectro.

In [ ]:
f, ax = plt.subplots()  

#ax.step(spec.spectral_axis, spec.flux)
#plt.xlabel('Rest Wavelength ({})'.format(spec.spectral_axis.unit)) 
#plt.ylabel('Flux ({})'.format(spec.flux.unit))
ax.plot(wave, flux)
plt.xlabel('Rest Wavelength (A)') 
plt.ylabel('Flux (cm/s/cm2/A)');

# Corrección por polvo en la Vía Láctea

INSTRUCCIÓN. Vamos a corregir la luz de las estrellas que se encuentran en la galaxia KISSR 298 por extinción debida al polvo en nuestra Galaxia.

Para esto, necesitamos el exceso de color debido al polvo en nuestra Galaxia, en la dirección de KISSR 298. 

Para calcular el exceso de color, E(B-V), primero encuentra la extinción Galáctica en las bandas Landolt B y V, yendo a la liga de abajo, introduciendo el nombre del objeto, luego yendo a la pestaña de Galactic Extinction, y finalmente yendo a la columna "Galactic Extinctions".
https://ned.ipac.caltech.edu/


In [ ]:
#Still being tested
#Fetch extinction table from the InfraRed Science Archive's (IRSA) Galactic Dust Reddening and Extinction 
# URL: https://irsa.ipac.caltech.edu/applications/DUST/index.html
from astroquery.irsa_dust import IrsaDust
#Have the `coords` module automatically look up the coordinates from the name of the source 
pos = coords.SkyCoord.from_name("KISSR 298")
table = IrsaDust.get_extinction_table(pos)
#Get the extinction values for the CTIO B and V filters
# This grabs the A_lambda values computed according to the Schlafly & Finkbeiner (2011) prescription,
#     which extends the Cardelli et al. (1989; CCM89) law to longer wavelengths.
A_B = table['A_SandF'][table['Filter_name'] == 'CTIO B'] # extinción Galáctica en la banda B
A_V = table['A_SandF'][table['Filter_name'] == 'CTIO V'] # extinción Galáctica en la banda V
ebvmw = A_B - A_V # exceso de color E(B-V) de la Vía Láctea (Milky Way)
print(ebvmw.unit, ebvmw.data[0])
ebvmw.quantity

In [ ]:
# B=0.062 # extinción Galáctica en la banda B
# V=0.047 # extinción Galáctica en la banda V
# ebvmw=B-V # exceso de color E(B-V) de la Vía Láctea (Milky Way)

Importamos un paquete necesario.

In [ ]:
import sys
# Install a pip package in the current Jupyter kernel if it isn't already installed
try:
  # Import this model: Cardelli, Clayton & Mathis (1989) with A_V = 1 and R_V = 3.1
  from dust_extinction.parameter_averages import CCM89
except:
  !{sys.executable} -m pip install dust_extinction
  # Import this model: Cardelli, Clayton & Mathis (1989) with A_V = 1 and R_V = 3.1
  from dust_extinction.parameter_averages import CCM89

In [ ]:
#from scipy.optimize import curve_fit

# Define the reddening model
ext = CCM89(Rv=3.1)

# Flujo corregido (unextinguished flux )
# uflux = flux / ext.extinguish(1/wave.to(u.micron), Ebv=ebvmw) 
uflux = flux / ext.extinguish(1/wave.to(u.micron), Ebv=0.1) #This should really be Ebv=ebvmw, but has been exaggerated for effect.

Graficamos el espectro antes y después de la corrección por enrojecimiento en la Vía Láctea.

In [ ]:
f, ax = plt.subplots()  

ax.step(spec.spectral_axis, spec.flux)
ax.plot(wave,uflux)
plt.xlabel('Observed Wavelength ({})'.format(spec.spectral_axis.unit)) 
plt.ylabel('Flux ({})'.format(spec.flux.unit))
_ = plt.ylim(0,5e-16)

# Corrección por corrimiento al rojo

INSTRUCCIÓN. Determina el corrimiento al rojo, z, de la galaxia.
Para esto, identifica las líneas espectrales de H-alfa y H-beta del hidrógeno. Si la galaxia no se estuviera ni alejando de ni acercando a nostros, las longitudes de onda en Agstroms de estas líneas fuertes del espectro estarían en 6563 y 4861, respectivamente. 

In [ ]:
z=0.04899 # Corrimiento al rojo encontrado

In [ ]:
wave_rest = wave / (1 + z)

Lo graficamos antes y después de la corrección por corrimiento al rojo.

In [ ]:
f, ax = plt.subplots()  

ax.plot(wave, uflux)
ax.plot(wave_rest, uflux)
plt.xlabel('Rest Wavelength ({})'.format(spec.spectral_axis.unit)) 
plt.ylabel('Flux ({})'.format(spec.flux.unit));